In [2]:
import argparse
from glob import glob

import numpy as np
from PIL import Image
from keras import backend as K
from keras import losses
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input, MaxPooling2D
from keras.layers import concatenate, Conv2D, Conv2DTranspose, Dropout, ReLU
from keras.models import Model
from keras.optimizers import Adam
from numpy import random
import tensorflow as tf
from random import randint
import keras
from keras_preprocessing import image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [0]:
batch_size = 16
input_shape = (64, 64)

def get_unet(do=0, activation=ReLU):
    inputs = Input((None, None, 3))
    conv1 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(inputs)))
    conv1 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv1)))
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(pool1)))
    conv2 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv2)))
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(pool2)))
    conv3 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv3)))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(pool3)))
    conv4 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv4)))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Dropout(do)(activation()(Conv2D(512, (3, 3), padding='same')(pool4)))
    conv5 = Dropout(do)(activation()(Conv2D(512, (3, 3), padding='same')(conv5)))

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(up6)))
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv6)))

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv9)))

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=['accuracy'])


    return model


def read_input(path):
    x = np.array(Image.open(path))/255.
    return x


def read_gt(path):
    x = np.array(Image.open(path))/255.
    return x[..., np.newaxis]


def random_crop(img, mask, crop_size=input_shape[0]):
    imgheight= img.shape[0]
    imgwidth = img.shape[1]
    i = randint(0, imgheight-crop_size)
    j = randint(0, imgwidth-crop_size)

    return img[i:(i+crop_size), j:(j+crop_size), :], mask[i:(i+crop_size), j:(j+crop_size)]


def gen(data):
    while True:
        repeat = 4
        index= random.choice(list(range(len(data))), batch_size//repeat)
        index = list(map(int, index))
        list_images_base = [read_input(data[i][0]) for i in index]
        list_gt_base = [read_gt(data[i][1]) for i in index]

        list_images = []
        list_gt = []

        for image, gt in zip(list_images_base, list_gt_base):

            for _ in range(repeat):
                image_, gt_ = random_crop(image.copy(), gt.copy())
                list_images.append(image_)
                list_gt.append(gt_)

        yield np.array(list_images), np.array(list_gt)


In [8]:
data = list(zip(sorted(glob('DRIVE/training/images/*.tif')),
                      sorted(glob('DRIVE/training/1st_manual/*.gif'))))


repeat = 4
index= random.choice(list(range(len(data))), batch_size//repeat)
index = list(map(int, index))
list_images_base = [read_input(data[i][0]) for i in index]
list_gt_base = [read_gt(data[i][1]) for i in index]

list_images = []
list_gt = []

for image, gt in zip(list_images_base, list_gt_base):

    for _ in range(repeat):
        image_, gt_ = random_crop(image.copy(), gt.copy())
        list_images.append(image_)
        list_gt.append(gt_)

list_images = np.array(list_images)
list_gt = np.array(list_gt)
len(data)

20

In [7]:
activation = globals()["ReLU"]
dropout = 0.1

train_data = np.array(list(zip(sorted(glob('DRIVE/training/images/*.tif')),
                      sorted(glob('DRIVE/training/1st_manual/*.gif')))))

seed = 7
np.random.seed(seed)
kf = KFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kf.split(train_data):
  model = get_unet(do=dropout, activation=activation)
  history = model.fit_generator(gen(train_data[train]), epochs=1, 
                              steps_per_epoch= 100*len(train_data)//batch_size)
  print("Model evaluation..")
  eval = model.evaluate_generator(gen(train_data[test]),steps=100*2//batch_size)
  print("Acc: ", eval[1], "testing: ", test)
  cvscores.append(eval[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
125/125 [==============================] - 23s 186ms/step - loss: 0.3544 - acc: 0.8875
Model evaluation..
Acc:  0.8936996459960938 testing:  [ 1 17]
Epoch 1/1
125/125 [==============================] - 19s 155ms/step - loss: 0.3746 - acc: 0.8895
Model evaluation..
Acc:  0.9097429911295573 testing:  [2 5]
Epoch 1/1
125/125 [==============================] - 21s 166ms/step - loss: 0.3429 - acc: 0.8912
Model evaluation..
Acc:  0.8987375895182291 testing:  [ 0 11]
Epoch 1/1
125/125 [==============================] - 20s 163ms/step - loss: 0.3485 - acc: 0.8925
Model evaluation..
Acc:  0.8933359781901041 testing:  [ 6 18]
Epoch 1/1
125/125 [==============================] - 21s 166ms/step - loss: 0.3528 - acc: 0.8876
Model evaluation..
Acc:  0.8955510

In [9]:
print("average: ", sum(cvscores)/10)

average:  0.894148254394531


In [0]:
!unzip DRIVE.zip